In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pymongo import MongoClient

from pylab import mpl
mpl.rcParams['font.sans-serif'] = ['SimHei']  #解决seaborn中文字体显示问题
plt.rc('figure', figsize=(10, 10))  #把plt默认的图片size调大一点
plt.rcParams["figure.dpi"] =mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题
%matplotlib inline

In [3]:
client = MongoClient("mongodb://localhost:27017/")
db = client['travel']

In [127]:
class DataCrawler(object):
    def __init__(self):
        self.cities = ["北京"] #list(pd.read_csv('city_data.csv')['city'])
        self.df = pd.DataFrame()
    def get_city_trip(self):
        for city in self.cities:
            print('正在爬取城市:{}的数据!'.format(city))
            res = requests.get('https://travelsearch.fliggy.com/async/queryItemResult.do?searchType='
                               'product&keyword={}&category=SCENIC&pagenum=1'.format(city))
            data = res.json()
            itemPagenum = data['data']['data'].get('itemPagenum')
            if itemPagenum is not None:
                page_count = itemPagenum['data']['count']
                print('共有{}页数据'.format(page_count))

                data_list = data['data']['data']['itemProducts']['data']['list'][0]['auctions']
                for ticket in data_list:
                    ticket['city'] = city
                    self.df = pd.concat([self.df, pd.json_normalize(ticket)])
                print('成功爬取城市:{}的第{}页数据!'.format(city, 1))

                if page_count > 1:
                    for page in range(2, page_count+1):
                        res = requests.get('https://travelsearch.fliggy.com/async/queryItemResult.do?searchType='
                                           'product&keyword={}&category=SCENIC&pagenum={}'.format(city, page))
                        data = res.json()
                        data_list = data['data']['data']['itemProducts']['data']['list'][0]['auctions']
                        for ticket in data_list:
                            ticket['city'] = city
                            self.df = pd.concat([self.df, pd.json_normalize(ticket)])
                        if page % 5 == 0:
                            print('成功爬取城市:{}的第{}页数据!'.format(city, page))
        
        self.df.index = np.arange(0, len(self.df), 1)

In [128]:
data_crawler = DataCrawler()

In [129]:
data_crawler.get_city_trip()

正在爬取城市:北京的数据!
共有26页数据
成功爬取城市:北京的第1页数据!
成功爬取城市:北京的第5页数据!
成功爬取城市:北京的第10页数据!
成功爬取城市:北京的第15页数据!
成功爬取城市:北京的第20页数据!
成功爬取城市:北京的第25页数据!


In [130]:
data_crawler.df

,trip_main_busness_type,src,city,fields.itemId,fields.title,fields.cityName,fields.comment,fields.picUrl,fields.soldRecent,fields.soldRecentNum,...,fields.pc_url,fields.distance,fields.longitude,fields.latitude,fields.p_score,fields.fromName,fields.features,fields.featuresNew,fields.tagList,fields.titleTagInfos
0,scenic,mix,北京,1818,故宫博物院,None,None,//img.alicdn.com/bao/uploaded/tfscom/TB1xZheXF...,已售3万+笔,29647,...,https://s.fliggy.com/scenic/detail.htm?sid=1818,None,116.397,39.9180,0,None,"[{'key': '一句话描述', 'text': '一层一层的朱门十分震撼，投在红墙上的斑...",None,"[{'text': '5A景区', 'icon': None, 'borderColor':...",[]
1,scenic,mix,北京,1735,颐和园,None,None,//img.alicdn.com/bao/uploaded/i3/1524402719400...,已售3591笔,3591,...,https://s.fliggy.com/scenic/detail.htm?sid=1735,None,116.273,39.9924,0,None,"[{'key': '一句话描述', 'text': '在苏州街穿梭，在昆明湖游荡，惬意使然，...",None,"[{'text': '5A景区', 'icon': None, 'borderColor':...",[]
2,scenic,mix,北京,1759,天坛公园,None,None,//img.alicdn.com/bao/uploaded/i4/1700802649294...,已售1853笔,1853,...,https://s.fliggy.com/scenic/detail.htm?sid=1759,None,116.414,39.8804,0,None,"[{'key': '一句话描述', 'text': '可以听听讲解，能学到不少祭坛的知识，很...",None,"[{'text': '5A景区', 'icon': None, 'borderColor':...",[]
3,scenic,mix,北京,1019,八达岭长城,None,None,//img.alicdn.com/bao/uploaded/i1/1700402634877...,已售550笔,550,...,https://s.fliggy.com/scenic/detail.htm?sid=1019,None,116.017,40.3562,0,None,"[{'key': '一句话描述', 'text': '居庸关的前哨，在明长城中保存最好，也最...",None,"[{'text': '5A景区', 'icon': None, 'borderColor':...",[]
4,scenic,mix,北京,210,北京欢乐谷,None,None,//img.alicdn.com/bao/uploaded/i1/O1CN01XgfmFx2...,已售967笔,967,...,https://s.fliggy.com/scenic/detail.htm?sid=210,None,116.495,39.8678,0,None,"[{'key': '一句话描述', 'text': '过山车一个比一个刺激，特别刺激，尖叫声...",None,"[{'text': '4A景区', 'icon': None, 'borderColor':...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1193,vocation,mix,北京,563985547911,[北京瑞麟湾温泉度假村-温泉票]北京瑞麟湾温泉度假村 温泉票,None,0,https://img.alicdn.com/bao/uploaded/i1/2750118...,None,0,...,https://traveldetail.fliggy.com/item.htm?id=56...,None,0.000,0.0000,0,None,"[{'key': None, 'text': '22:58前可订今日票', 'color':...","[{'key': None, 'text': '22:58前可订今日票', 'color':...",[],[]
1194,vocation,mix,北京,609736173190,[石京龙滑雪场-4小时滑雪票]北京万科石京龙滑雪场4小时滑雪,None,0,https://img.alicdn.com/bao/uploaded/i3/2292851...,None,0,...,https://traveldetail.fliggy.com/item.htm?id=60...,None,0.000,0.0000,0,None,"[{'key': None, 'text': '15:55前可订今日票', 'color':...","[{'key': None, 'text': '15:55前可订今日票', 'color':...",[],[]
1195,vocation,mix,北京,608358398199,[赛纳河休闲商务会馆（立水桥店）-大门票]北京,None,0,https://img.alicdn.com/bao/uploaded/i1/7114205...,None,0,...,https://traveldetail.fliggy.com/item.htm?id=60...,None,0.000,0.0000,0,None,"[{'key': None, 'text': '17:00前可订今日票', 'color':...","[{'key': None, 'text': '17:00前可订今日票', 'color':...",[],[]
1196,vocation,mix,北京,583700698271,[莲花山滑雪场-半天滑雪票]北京 莲花山滑雪场 4小时滑雪,None,7,https://img.alicdn.com/bao/uploaded/i2/2750118...,None,0,...,https://traveldetail.fliggy.com/item.htm?id=58...,None,0.000,0.0000,0,None,"[{'key': None, 'text': '14:40前可订今日票', 'color':...","[{'key': None, 'text': '14:40前可订今日票', 'color':...",[],[]


In [ ]:
data_crawler.df['fields.soldRecentNum']

In [120]:
data_crawler.df['fields.discountPrice']

0      59.5
1       218
2        76
3       205
4        75
       ... 
289    39.9
290      62
291      30
292      98
293      53
Name: fields.discountPrice, Length: 294, dtype: object

In [131]:
data_crawler.df['fields.discountPrice'] = data_crawler.df['fields.discountPrice'].astype(float)
data_crawler.df['fields.price'] = data_crawler.df['fields.price'].astype(float)

In [132]:
data_crawler.df['discount'] = np.round((1 - (data_crawler.df['fields.price'] - 
                                        data_crawler.df['fields.discountPrice'])
                                       / data_crawler.df['fields.price']) * 10, 1)

In [134]:
set(data_crawler.df['discount'].values)

{3.2,
 7.5,
 7.6,
 7.7,
 7.9,
 8.0,
 8.2,
 8.5,
 8.6,
 8.8,
 9.0,
 9.1,
 9.2,
 9.3,
 9.4,
 9.5,
 9.6,
 9.7,
 9.8,
 9.9,
 10.0}

In [116]:
np.sum(np.array(data_crawler.df['fields.soldRecentNum'].values, dtype=int))

3492

In [111]:
np.sum(np.array(data_crawler.df['fields.soldRecentNum'].values, dtype=int))

96578

In [103]:
data_crawler.df.columns

Index(['trip_main_busness_type', 'src', 'city', 'fields.itemId',
       'fields.title', 'fields.cityName', 'fields.comment', 'fields.picUrl',
       'fields.soldRecent', 'fields.soldRecentNum', 'fields.sold365',
       'fields.discountPrice', 'fields.type', 'fields.subTitle',
       'fields.shortInfo', 'fields.scm', 'fields.price',
       'fields.itemTotalScore', 'fields.h5_url', 'fields.native_url',
       'fields.pc_url', 'fields.distance', 'fields.longitude',
       'fields.latitude', 'fields.p_score', 'fields.fromName',
       'fields.features', 'fields.featuresNew', 'fields.tagList',
       'fields.titleTagInfos'],
      dtype='object')

In [43]:
data['data']['data']['itemProducts']['data']['list'][0]['auctions'][0]['fields']

{'itemId': '1818',
 'title': '故宫博物院',
 'cityName': None,
 'comment': None,
 'picUrl': '//img.alicdn.com/bao/uploaded/tfscom/TB1xZheXFmWBuNjSspdSuvugXXa',
 'soldRecent': '已售3万+笔',
 'soldRecentNum': '29647',
 'sold365': None,
 'discountPrice': '38.5',
 'type': None,
 'subTitle': None,
 'shortInfo': None,
 'scm': None,
 'price': '38.5',
 'itemTotalScore': None,
 'h5_url': None,
 'native_url': None,
 'pc_url': 'https://s.fliggy.com/scenic/detail.htm?sid=1818',
 'distance': None,
 'longitude': 116.397,
 'latitude': 39.918,
 'p_score': '0',
 'fromName': None,
 'features': [{'key': '一句话描述',
   'text': '一层一层的朱门十分震撼，投在红墙上的斑影拍照极美。',
   'color': None,
   'iconUrl': None}],
 'featuresNew': None,
 'tagList': [{'text': '5A景区',
   'icon': None,
   'borderColor': '#FCE9B8',
   'textColor': '#FCA500',
   'bgColor': None,
   'pormotion': False,
   'className': None,
   'tagType': {'name': 'NONE'}},
  {'text': '今日可用',
   'icon': None,
   'borderColor': '#FCE9B8',
   'textColor': '#FCA500',
   'bgColor': 

In [45]:
pd.DataFrame(data['data']['data']['itemProducts']['data']['list'][0]['auctions'])

,trip_main_busness_type,src,fields,_id
0,scenic,mix,"{'itemId': '1818', 'title': '故宫博物院', 'cityName...",5e402f6928ba754ac330b1aa
1,scenic,mix,"{'itemId': '1735', 'title': '颐和园', 'cityName':...",NaN
2,scenic,mix,"{'itemId': '1759', 'title': '天坛公园', 'cityName'...",NaN
3,scenic,mix,"{'itemId': '1019', 'title': '八达岭长城', 'cityName...",NaN
4,scenic,mix,"{'itemId': '210', 'title': '北京欢乐谷', 'cityName'...",NaN
5,scenic,mix,"{'itemId': '17531', 'title': '北京海洋馆', 'cityNam...",NaN
6,scenic,mix,"{'itemId': '1631', 'title': '圆明园', 'cityName':...",NaN
7,scenic,mix,"{'itemId': '1635', 'title': '北京野生动物园', 'cityNa...",NaN
8,scenic,mix,"{'itemId': '4028', 'title': '古北水镇', 'cityName'...",NaN
9,scenic,mix,"{'itemId': '1664', 'title': '恭王府', 'cityName':...",NaN


In [6]:
import requests
import pandas as pd

In [7]:
city = '北京'
res = requests.get('https://travelsearch.fliggy.com/async/queryItemResult.do?searchType='
                   'product&keyword={}&category=SCENIC&pagenum=1'.format(city))
data = res.json()

In [36]:
data['data']['data']['itemProducts']['data']['list'][0]['auctions'][0]

{'trip_main_busness_type': 'scenic',
 'src': 'mix',
 'fields': {'itemId': '1818',
  'title': '故宫博物院',
  'cityName': None,
  'comment': None,
  'picUrl': '//img.alicdn.com/bao/uploaded/tfscom/TB1xZheXFmWBuNjSspdSuvugXXa',
  'soldRecent': '已售3万+笔',
  'soldRecentNum': '29647',
  'sold365': None,
  'discountPrice': '38.5',
  'type': None,
  'subTitle': None,
  'shortInfo': None,
  'scm': None,
  'price': '38.5',
  'itemTotalScore': None,
  'h5_url': None,
  'native_url': None,
  'pc_url': 'https://s.fliggy.com/scenic/detail.htm?sid=1818',
  'distance': None,
  'longitude': 116.397,
  'latitude': 39.918,
  'p_score': '0',
  'fromName': None,
  'features': [{'key': '一句话描述',
    'text': '一层一层的朱门十分震撼，投在红墙上的斑影拍照极美。',
    'color': None,
    'iconUrl': None}],
  'featuresNew': None,
  'tagList': [{'text': '5A景区',
    'icon': None,
    'borderColor': '#FCE9B8',
    'textColor': '#FCA500',
    'bgColor': None,
    'pormotion': False,
    'className': None,
    'tagType': {'name': 'NONE'}},
   {'tex

In [ ]:
itemPagenum = data['data']['data'].get('itemPagenum')
            if itemPagenum is not None:
                page_count = itemPagenum['data']['count']

                data_list = data['data']['data']['itemProducts']['data']['list'][0]['auctions']
                for ticket in data_list:
                    ticket['city'] = city
                    self.col.insert_one(ticket)
                print('成功爬取城市:{}的第{}页数据!'.format(city, 1))

                if page_count > 1:
                    for page in range(2, page_count+1):
                        res = requests.get('https://travelsearch.fliggy.com/async/queryItemResult.do?searchType='
                                           'product&keyword={}&category=SCENIC&pagenum={}'.format(city, page))
                        data = res.json()
                        data_list = data['data']['data']['itemProducts']['data']['list'][0]['auctions']
                        for ticket in data_list:
                            ticket['city'] = city
                            self.col.insert_one(ticket)
                        print('成功爬取城市:{}的第{}页数据!'.format(city, page))

In [ ]:
conn = MongoClient(host='127.0.0.1', port=27017)  # 实例化MongoClient
db = conn.get_database('Laborday')  

col = db.get_collection('ticket') # 连接到集合ticket
mon_data = col.find()  # 查询这个集合下的所有记录